In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiverbaselinezyz/cbt_0.model
/kaggle/input/optiverbaselinezyz/lgb_3.model
/kaggle/input/optiverbaselinezyz/cbt_2.model
/kaggle/input/optiverbaselinezyz/lgb_0.model
/kaggle/input/optiverbaselinezyz/lgb_4.model
/kaggle/input/optiverbaselinezyz/xgb_1.model
/kaggle/input/optiverbaselinezyz/xgb_2.model
/kaggle/input/optiverbaselinezyz/cbt_3.model
/kaggle/input/optiverbaselinezyz/xgb_3.model
/kaggle/input/optiverbaselinezyz/lgb_1.model
/kaggle/input/optiverbaselinezyz/xgb_4.model
/kaggle/input/optiverbaselinezyz/xgb_0.model
/kaggle/input/optiverbaselinezyz/lgb_2.model
/kaggle/input/optiverbaselinezyz/cbt_1.model
/kaggle/input/optiverbaselinezyz/cbt_4.model
/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-tradin

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [44]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [45]:
df['stock_id'] = df['stock_id'].astype(np.int32)
df['date_id'] = df['date_id'].astype(np.int32)
df['seconds_in_bucket'] = df['seconds_in_bucket'].astype(np.int32)
df['imbalance_buy_sell_flag'] = df['imbalance_buy_sell_flag'].astype(np.int32)
df.drop(['time_id','row_id'],axis=1,inplace=True)

In [46]:
df['stock_id'] = df['stock_id'].astype(np.int32)
df['date_id'] = df['date_id'].astype(np.int32)
df['seconds_in_bucket'] = df['seconds_in_bucket'].astype(np.int32)
df['imbalance_buy_sell_flag'] = df['imbalance_buy_sell_flag'].astype(np.int32)

In [47]:
stock_id = 10
date_id = 100

df.groupby(['stock_id','date_id'])['seconds_in_bucket'].count().apply(lambda x: x==55).any()

True

In [48]:
df = df.dropna(subset=['wap'])

In [49]:
df.isnull().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                   0
imbalance_buy_sell_flag          0
reference_price                  0
matched_size                     0
far_price                  2894122
near_price                 2856960
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
dtype: int64

In [50]:
import pandas as pd 
def generate_features(df):
    features = ['seconds_in_bucket','imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_
    
    return df[features]

In [51]:
final = generate_features(df)
final.head()

,seconds_in_bucket,imbalance_buy_sell_flag,imbalance_size,matched_size,bid_size,ask_size,reference_price,far_price,near_price,ask_price,...,ask_price_near_price_imb,bid_price_reference_price_imb,bid_price_far_price_imb,bid_price_near_price_imb,bid_price_ask_price_imb,wap_reference_price_imb,wap_far_price_imb,wap_near_price_imb,wap_ask_price_imb,wap_bid_price_imb
0,0,1,3180602.69,13380276.64,60651.50,8493.03,0.999812,NaN,NaN,1.000026,...,NaN,0.000000,NaN,NaN,-0.000107,0.000094,NaN,NaN,-0.000013,9.400884e-05
1,0,-1,166603.91,1642214.25,3233.04,20605.09,0.999896,NaN,NaN,1.000660,...,NaN,0.000000,NaN,NaN,-0.000382,0.000052,NaN,NaN,-0.000330,5.200270e-05
2,0,-1,302879.87,1819368.03,37956.00,18995.00,0.999561,NaN,NaN,1.000298,...,NaN,-0.000079,NaN,NaN,-0.000448,0.000220,NaN,NaN,-0.000149,2.985891e-04
3,0,-1,11917682.27,18389745.62,2324.90,479032.40,1.000171,NaN,NaN,1.000214,...,NaN,-0.000086,NaN,NaN,-0.000107,-0.000085,NaN,NaN,-0.000107,5.000003e-07
4,0,-1,447549.96,17860614.95,16485.54,434.10,0.999532,NaN,NaN,1.000016,...,NaN,-0.000069,NaN,NaN,-0.000311,0.000234,NaN,NaN,-0.000008,3.030918e-04


In [52]:
after_355 = final[final['seconds_in_bucket'] > 300]
before_355 = final[final['seconds_in_bucket'] <= 300]

In [53]:
after_355['seconds_in_bucket'].unique()
after_355.head()

,seconds_in_bucket,imbalance_buy_sell_flag,imbalance_size,matched_size,bid_size,ask_size,reference_price,far_price,near_price,ask_price,...,ask_price_near_price_imb,bid_price_reference_price_imb,bid_price_far_price_imb,bid_price_near_price_imb,bid_price_ask_price_imb,wap_reference_price_imb,wap_far_price_imb,wap_near_price_imb,wap_ask_price_imb,wap_bid_price_imb
5921,310,0,0.00,26670118.88,22767.64,18664.00,0.999919,0.999919,0.999919,0.999919,...,0.000000,-0.000054,-0.000054,-0.000054,-0.000054,-0.000024,-0.000024,-0.000024,-0.000024,0.000030
5922,310,-1,125761.28,3358625.96,1701.20,3234.18,0.999661,0.988849,0.999132,1.000249,...,0.000559,0.000000,0.005437,0.000265,-0.000294,0.000102,0.005539,0.000366,-0.000192,0.000102
5923,310,0,0.00,4501991.15,94900.00,83975.58,0.999719,0.999719,0.999719,1.000509,...,0.000395,-0.000106,-0.000106,-0.000106,-0.000500,0.000161,0.000161,0.000161,-0.000234,0.000266
5924,310,1,2414178.67,41593666.66,23236.00,2556.51,0.999655,1.002537,1.001978,0.999655,...,-0.001161,-0.000108,-0.001547,-0.001268,-0.000108,-0.000011,-0.001450,-0.001171,-0.000011,0.000097
5925,310,1,3042413.68,38710684.23,35414.75,14460.00,0.999325,1.007894,1.007894,0.999325,...,-0.004269,-0.000173,-0.004442,-0.004442,-0.000173,-0.000050,-0.004319,-0.004319,-0.000050,0.000123


In [54]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

def test_train_val_split(final):
    train_set = []
    val_set = []
    test_set = []
    final['date_id'] = df['date_id']
    final['target'] = df['target']

    final['Train'] = "Train"
    final['Train'] = final['date_id'].apply(lambda x: "Val" if ((x % 4 ==0) | (x % 5 == 0)) else ("Test" if (x % 6 == 0) else "Train"))

    print(len(final[final['Train'] == 'Train'])) #2787950
    print(len(final[final['Train'] == 'Val'])) #2101605
    print(len(final[final['Train'] == 'Test'])) # 348425

    X_train = final[final['Train'] == 'Train'].drop(columns=['target','Train'])
    y_train = final[final['Train'] == 'Train']['target']

    X_test = final[final['Train'] == 'Test'].drop(columns=['target','Train'])
    y_test = final[final['Train'] == 'Test']['target']

    X_validation = final[final['Train'] == 'Val'].drop(columns=['target','Train'])
    y_validation = final[final['Train'] == 'Val']['target']

    final.drop(['target','Train'],axis=1,inplace=True)
    X_train.drop('date_id',axis=1,inplace=True)
    X_test.drop('date_id',axis=1,inplace=True)
    X_validation.drop('date_id',axis=1,inplace=True)
    print(X_train.shape)

    return X_train,y_train,X_test,y_test,X_validation,y_validation


def model_predict(X_train,y_train,X_test,y_test,X_validation,y_validation):
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_validation)
    print("Validation Score",mean_absolute_error(y_pred,y_validation))

    y_pred = model.predict(X_test)
    print("Test Score",mean_absolute_error(y_pred,y_test))

    return model

In [55]:
X_train,y_train,X_test,y_test,X_validation,y_validation = test_train_val_split(after_355)
model_a = model_predict(X_train,y_train,X_test,y_test,X_validation,y_validation)

X_train,y_train,X_test,y_test,X_validation,y_validation = test_train_val_split(before_355)
model_b = model_predict(X_train,y_train,X_test,y_test,X_validation,y_validation)

1216560
916992
152016
(1216560, 29)
Validation Score 5.601981540269374
Test Score 5.433828119872766
1571390
1184448
196354
(1571390, 29)
Validation Score 6.81654962592175
Test Score 6.705637318175443


In [57]:
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
test = generate_features(test)
print(test.shape)

after_355 = test[test['seconds_in_bucket'] > 300]
before_355 = test[test['seconds_in_bucket'] <= 300]

target_a355 = model_a.predict(after_355)
target_b355 = model_b.predict(before_355)

target_a355 = pd.Series(target_a355, index=after_355.index)
target_b355 = pd.Series(target_b355, index=before_355.index)

target_pred = pd.concat([target_a355,target_b355],axis=0)
final = test.merge(target_pred.to_frame(),left_index=True, right_index=True)
final.rename( columns={0 :'target'},inplace=True)

(33000, 29)


In [65]:
final_sub = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv")
final_submission = final[['target']].merge(final_sub['row_id'], left_index=True, right_index=True)
final_submission = final_submission[['row_id','target']]
final_submission.iloc[:3,:].to_csv("submission.csv",mode='a+')